In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,541 kB]
Get:14 http://archive.ubuntu.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
# Spark context
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql import Row
from pyspark.sql.functions import to_date, when, col, lit, concat, year, month, when, avg


spark = SparkSession.builder.appName("Lab 04; Movies").getOrCreate()

# Initialize Spark session
df = spark.read \
    .option("inferSchema", "true") \
    .json("/content/drive/MyDrive/movies.json") \

# Show the updated DataFrame with the correct 'order_date' column
#df.show()

# Print the schema to verify the new 'order_date' column is of DateType
df.printSchema()


root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)



In [66]:
"""
Calculate Profit Margin: Compute the profit margin for each movie
by subtracting the Production Budget from the Worldwide Gross
and then dividing by the Production Budget
"""

df_movies_1 = df.withColumn('profit_margin', \
                    (df['Production_Budget'] - df["Worldwide_Gross"]) / df['Production_Budget'] ) \
                    .select(df['Title'], df['Production_Budget'], df["Worldwide_Gross"], 'profit_margin')

df_movies_1.show()

+--------------------+-----------------+---------------+--------------------+
|               Title|Production_Budget|Worldwide_Gross|       profit_margin|
+--------------------+-----------------+---------------+--------------------+
|      The Land Girls|          8000000|         146083|         0.981739625|
|First Love, Last ...|           300000|          10876|  0.9637466666666666|
|I Married a Stran...|           250000|         203134|            0.187464|
|Let's Talk About Sex|           300000|         373615|-0.24538333333333334|
|                Slam|          1000000|        1087521|           -0.087521|
| Mississippi Mermaid|          1600000|        2624551|        -0.640344375|
|           Following|             6000|          44705|  -6.450833333333334|
|             Foolish|          1600000|        6026908|          -2.7668175|
|             Pirates|         40000000|        6341825|         0.841454375|
|     Duel in the Sun|          6000000|       20400000|        

In [67]:
"""
Determine if a Movie is a Box Office Hit: Create a column
Box Office Hit that categorizes movies as ”Hit” if
Worldwide Gross is greater than twice the Production Budget,
and ”Flop” otherwise.
"""

movies_df = df.withColumn(
    "Box Office Hit",
    when(col("Worldwide_Gross") > (2 * col("Production_Budget")), "Hit").otherwise("Flop")
)

movies_df.select("Title", "Worldwide_Gross", "Production_Budget", "Box Office Hit").show()

+--------------------+---------------+-----------------+--------------+
|               Title|Worldwide_Gross|Production_Budget|Box Office Hit|
+--------------------+---------------+-----------------+--------------+
|      The Land Girls|         146083|          8000000|          Flop|
|First Love, Last ...|          10876|           300000|          Flop|
|I Married a Stran...|         203134|           250000|          Flop|
|Let's Talk About Sex|         373615|           300000|          Flop|
|                Slam|        1087521|          1000000|          Flop|
| Mississippi Mermaid|        2624551|          1600000|          Flop|
|           Following|          44705|             6000|           Hit|
|             Foolish|        6026908|          1600000|           Hit|
|             Pirates|        6341825|         40000000|          Flop|
|     Duel in the Sun|       20400000|          6000000|           Hit|
|           Tom Jones|       37600000|          1000000|        

In [68]:
"""
Convert Release Date to a Standard Format: Convert the
Release Date from DD-MMM-YY to a DateType column and
extract the year in a new column.
"""

## We format the release date
df = df.withColumn('Release_Date_Format',
          when(col('Release_Date').rlike("^\d{4}-\d{2}-\d{2}$"), to_date(col('Release_Date'), 'yyyy-MM-dd'))
          .when(col('Release_Date').rlike("^\d{2}/\d{2}/\d{4}$"), to_date(col('Release_Date'), 'MM/dd/yyyy)'))
          .when(col("Release_Date").rlike(r'^[0-9]{2}-[A-Za-z]{3}-[0-9]{2}$'), to_date(col("Release_Date"), "dd-MMM-yy"))
          .when(col("Release_Date").rlike(r'^[0-9]{1}-[A-Za-z]{3}-[0-9]{2}$'), to_date(col("Release_Date"), "d-MMM-yy"))
          .otherwise(None) # Handle unexpected formats with None
      )

## We extract the year from the release year
df = df.withColumn('Release_Year',
                   when(year(df['Release_Date_Format']) > 2000, year(df['Release_Date_Format']) - 100)
                  .otherwise(year(df['Release_Date_Format']))
          )


df.select('Title', 'Release_Date', 'Release_Date_Format', 'Release_Year').show()


+--------------------+------------+-------------------+------------+
|               Title|Release_Date|Release_Date_Format|Release_Year|
+--------------------+------------+-------------------+------------+
|      The Land Girls|   12-Jun-98|         2098-06-12|        1998|
|First Love, Last ...|    7-Aug-98|         2098-08-07|        1998|
|I Married a Stran...|   28-Aug-98|         2098-08-28|        1998|
|Let's Talk About Sex|   11-Sep-98|         2098-09-11|        1998|
|                Slam|    9-Oct-98|         2098-10-09|        1998|
| Mississippi Mermaid|   15-Jan-99|         2099-01-15|        1999|
|           Following|    4-Apr-99|         2099-04-04|        1999|
|             Foolish|    9-Apr-99|         2099-04-09|        1999|
|             Pirates|    1-Jul-86|         2086-07-01|        1986|
|     Duel in the Sun|   31-Dec-46|         2046-12-31|        1946|
|           Tom Jones|    7-Oct-63|         2063-10-07|        1963|
|             Oliver!|   11-Dec-68

In [69]:
#Create a new column: IMDB Rating Category to categorize movies
#based on their IMDB Rating: ”High” if the rating is 7.0 or above,
#"Medium” if between 5.0 and 6.9, and ”Low” if below 5.0.

df_movies = df.withColumn(
    "IMDB_Rating_Category",
    when(df.IMDB_Rating >= 7.0, "High")
    .when((df.IMDB_Rating >= 5.0) & (df.IMDB_Rating < 7.0), "Medium")
    .otherwise("Low")
)

df_selected = df_movies.select("IMDB_Rating_Category")
df_selected.show()


+--------------------+
|IMDB_Rating_Category|
+--------------------+
|              Medium|
|              Medium|
|              Medium|
|                 Low|
|                 Low|
|                 Low|
|                High|
|                 Low|
|              Medium|
|                High|
|                High|
|                High|
|                High|
|                 Low|
|              Medium|
|                 Low|
|                High|
|              Medium|
|                 Low|
|                High|
+--------------------+
only showing top 20 rows



In [70]:

"""
Calculate Average IMDB Rating for Each Distributor: Calculate the
average IMDB Rating for each Distributor and create a new
DataFrame with this information.
"""

average_imbd_rating = df \
                      .groupBy('Distributor') \
                      .agg(avg('IMDB_Rating')) \
                      .withColumnRenamed('avg(IMDB_Rating)', 'Average_IMDB_Rating') \

average_imbd_rating.show()


+--------------------+-------------------+
|         Distributor|Average_IMDB_Rating|
+--------------------+-------------------+
|Oscilloscope Pict...|                6.2|
|               Savoy|  6.533333333333334|
|             Embassy|               NULL|
|         Fader Films|                6.5|
|       October Films|                6.4|
|              Strand|  6.322222222222223|
|             Trimark|  5.085714285714286|
|              Matson|                7.3|
|      Cinema Service|                7.1|
| Weinstein/Dimension| 5.7250000000000005|
|Providence Entert...|                3.3|
|      Big Fat Movies|                3.2|
|     Newmarket Films| 7.4142857142857155|
|    First Run/Icarus|               NULL|
|  Cloud Ten Pictures|                5.0|
|   Outrider Pictures|                7.0|
|     Slowhand Cinema|                5.3|
|   Paramount Vantage|                7.2|
|   Magnolia Pictures|             6.4625|
|  Kino International| 6.0200000000000005|
+----------